In [145]:
print(f"To check the SSL certificate expiration date, we will use the OpenSSL library. This package provides a high-level interface to the functions in the OpenSSL library.")

To check the SSL certificate expiration date, we will use the OpenSSL library. This package provides a high-level interface to the functions in the OpenSSL library.


In [146]:
#!pip install pyopenssl

In [147]:
from datetime import datetime, date
import OpenSSL
import ssl
import os
import json
import pandas as pd
from dotenv import load_dotenv
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

In [148]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables

True

In [149]:
source_es_host = "http://localhost:9211"
# source_es_host = os.getenv('ES_DEV_V8_HOST')

In [150]:
source_es_host = source_es_host.replace("http://","").replace("https://","")
source_es_hostname = str(source_es_host.split(':')[0])
source_es_port = str(source_es_host.split(':')[1])

In [151]:
print(source_es_hostname, source_es_port)

localhost 9211


In [152]:
response_dict = {}

In [153]:
def get_ssl_certs_hosts(source_es_hostname, source_es_port):
    ''' check if the ES cluster has any ssl certs and get the expired date '''
    try:
        cert=ssl.get_server_certificate((source_es_hostname, source_es_port))
        x509 = OpenSSL.crypto.load_certificate(OpenSSL.crypto.FILETYPE_PEM, cert)
        bytes=x509.get_notAfter()
        # print(bytes)
        timestamp = bytes.decode('utf-8')
        # print (datetime.strptime(timestamp, '%Y%m%d%H%M%S%z').date().isoformat())
        # print(datetime.strptime(timestamp, '%Y%m%d%H%M%S%z'))
        ssl_expire_date = datetime.strptime(timestamp, '%Y%m%d%H%M%S%z')
        ssl_expire_date = "{}-{}-{}".format(str(ssl_expire_date.year).zfill(2), str(ssl_expire_date.month).zfill(2), str(ssl_expire_date.day).zfill(2))
    
        response_dict.update({"ssl_certs_expire_date" : ssl_expire_date})
        response_dict.update({"ssl_certs_expire_yyyymmdd" : int(ssl_expire_date.replace("-",""))})
    except Exception as e:
        response_dict.update({"ssl_certs_expire_date" : 'no_ssl_certs'})
        response_dict.update({"ssl_certs_expire_yyyymmdd" : 0})
        print(e)
    finally:
        return response_dict

In [154]:
response_dict = get_ssl_certs_hosts(source_es_hostname, source_es_port)

[WinError 10061] No connection could be made because the target machine actively refused it


In [155]:
print(json.dumps(response_dict, indent=2)) 

{
  "ssl_certs_expire_date": "no_ssl_certs",
  "ssl_certs_expire_yyyymmdd": 0
}


In [156]:
response_dict_env = {}

In [157]:
env, cluster, host, service, ssl_certs_expire_date = [], [], [], [], []
all_env_ssl_df = {}

In [158]:
for k, v in response_dict_env.items():
    if v.get("ssl_certs_expire_yyyymmdd") > 0:
        # print(k.rsplit("_",2))
        env_name = k.rsplit("_",2)[0]
        env.append(env_name)
        cluster.append(k)
        host.append(v.get("host")) 
        ssl_certs_expire_date.append(v.get("ssl_certs_expire_date")) 
        if 'es' in k:
            service.append('Elasticsearch')
        elif 'spark' in k:
            service.append('Spark')

all_env_ssl_df.update({'env' : env})
all_env_ssl_df.update({'cluster' : cluster})
all_env_ssl_df.update({'host' : host})
all_env_ssl_df.update({'service' : service})
all_env_ssl_df.update({'ssl_certs_expired_date' : ssl_certs_expire_date})

In [159]:
''' Compare the ES indices between source cluster and target cluster '''
df = pd.DataFrame.from_dict(all_env_ssl_df)
# df.head(10)
display(df)

,env,cluster,host,service,ssl_certs_expired_date


In [160]:
# Define a styling function
def highlight_failed_rows(row):
    # Apply red background if 'Status' is 'Fail'
    if row['Age'] < 30:
        return ['background-color: red; font-weight: bold'] * len(row)
    else:
        return [''] * len(row)

In [161]:
''' sample '''
''' https://docs.kanaries.net/ko/topics/Pandas/pandas-add-column '''
data = {
    'Name': ['Alice', 'Bob', 'Charlie', 'David'],
    'Age': [25, 30, 35, 40]
}
 
df = pd.DataFrame(data)
# Apply the styling function to the DataFrame
df = df.style.apply(highlight_failed_rows, axis=1)
# df.head(10)
display(df)

,Name,Age
0,Alice,25
1,Bob,30
2,Charlie,35
3,David,40


In [162]:
print(f"** The number of days remaining until the certificate expires from today's date.**")

** The number of days remaining until the certificate expires from today's date.**


In [163]:
def subtract_dates_yyyymmdd(date_str1, date_str2):
    """
    Calculates the number of days between two dates in YYYYMMDD format.

    Args:
        date_str1 (str): The first date string in YYYYMMDD format.
        date_str2 (str): The second date string in YYYYMMDD format.

    Returns:
        int: The number of days between the two dates.
    """
    date1 = datetime.strptime(date_str1, "%Y-%m-%d")
    date2 = datetime.strptime(date_str2, "%Y-%m-%d")
    
    time_difference = date1 - date2
    return time_difference.days

In [164]:
# Example usage:
date1_str = "2035-08-09"
# date2_str = "2023-01-15"
date2_str = date.today().strftime("%Y-%m-%d")
days_difference = subtract_dates_yyyymmdd(date1_str, date2_str)

In [165]:
# threshold_days_of_expiration_date = 3650
threshold_days_of_expiration_date = 60

In [166]:
if int(days_difference) < threshold_days_of_expiration_date:
    print(f"[Need to re-generation a ssl certificate] The difference between {date1_str} and {date2_str} is {days_difference} days.")
else:
    print(f"[Ok] The difference between {date1_str} and {date2_str} is {days_difference} days.")

[Ok] The difference between 2035-08-09 and 2025-10-09 is 3591 days.


In [167]:
data = {}
expiration_date_list, today_list, diff_days_list,  diff_check_list = [], [], [], []

In [168]:
def get_dataframe_expiration_date(env, service, host, expiration_date_list, today_list, diff_days_list, threshold_expiration_date):
    ''' Check the status of the expiration date of certificate '''
    # Define a styling function
    def highlight_failed_rows(row):
        # Apply red background if 'Status' is 'Fail'
        if row['diff_days'] < threshold_days_of_expiration_date:
            return ['background-color: red; font-weight: bold'] * len(row)
        else:
            return [''] * len(row)
        
    diff_check_list = []
    for v in diff_days_list:
        if int(v) < threshold_expiration_date:
            diff_check_list.append('Red')
            continue
        diff_check_list.append('Green')    
            
    data.update({
        'env' : env,
        'service' : service,
        'host' : host,
        'expiration_date': expiration_date_list,
        'today': today_list,
        'diff_days': diff_days_list,
        'status' : diff_check_list
    })
     
    df = pd.DataFrame(data)
    # print(df.head(10))
    # Apply the styling function to the DataFrame
    styled_df = df.style.apply(highlight_failed_rows, axis=1)
    # display(df)
    display(styled_df)

In [169]:
# expiration_date_list.append(date1_str)
# today_list.append(days_difference)
# diff_days_list.append(days_difference)
for v in all_env_ssl_df.get("ssl_certs_expired_date"):
    # print(v)
    expiration_date_list.append(v)
    today_list.append(date.today().strftime("%Y-%m-%d"))
    diff_days_list.append(subtract_dates_yyyymmdd(v, date.today().strftime("%Y-%m-%d")))

In [170]:
''' Test call to get the days of expiration date '''
get_dataframe_expiration_date(
                              all_env_ssl_df.get("env"),
                              all_env_ssl_df.get("service"),
                              all_env_ssl_df.get("host"),
                              expiration_date_list, 
                              today_list, 
                              diff_days_list, 
                              threshold_days_of_expiration_date)

,env,service,host,expiration_date,today,diff_days,status


In [171]:
# Write DataFrame to CSV File with Default params.
# filename = f"./ssl_certs_all_env_df.csv"
# if not df.empty:
#     df.to_csv(f"{filename}")
#     print(f"{filename} was created..")

In [172]:
HTML('<pre style="background-color: #A52A2A; color: white;">Job is being performed correctly</pre>')